In [1]:
!pip install mlforecast
!pip install window_ops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 15.8 MB/s eta 0:00:00


In [2]:
###### Feature Engineering ######
import pandas as pd
import numpy as np
from window_ops.rolling import rolling_mean, rolling_max, rolling_min

###### Load the API  ######
import requests
import gzip
import io

###### Forecasting ######
from mlforecast import MLForecast
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
import lightgbm as lgb



###### Visualize ######
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from window_ops.rolling import rolling_mean, rolling_max, rolling_min
from sklearn.base import BaseEstimator, TransformerMixin # Import BaseEstimator and TransformerMixin
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder

###### Load the API  ######
import requests
import gzip
import io

###### Forecasting ######
from mlforecast import MLForecast
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
import lightgbm as lgb


###### Visualize ######
import matplotlib.pyplot as plt
import seaborn as sns

armamos funciones basicas

In [3]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
import requests
import gzip
import io
import matplotlib.pyplot as plt
import seaborn as sns


def calculate_error(forecast: np.ndarray, actual: np.ndarray) -> float:
    """
    Calcula el Total Forecast Error:

        ∑ |actual_i - forecast_i|
        -------------------------
           ∑ actual_i

    Parámetros
    ----------
    forecast : np.ndarray
        Array de valores pronosticados (ForecastSales).
    actual : np.ndarray
        Array de valores reales (ActualSales).

    Devuelve
    -------
    float
        El error total de pronóstico.
    """
    # Asegurarnos de que sean floats para evitar divisiones enteras
    forecast = forecast.astype(float)
    actual   = actual.astype(float)

    # Numerador: suma de errores absolutos
    numerador = np.sum(np.abs(actual - forecast))
    # Denominador: suma de ventas reales
    denominador = np.sum(actual)

    # Evitar división por cero
    if denominador == 0:
        raise ValueError("La suma de actual (denominador) es cero.")

    return numerador / denominador



def cargar_dataset(url, sep='\t', header='infer'):
    """
    Lee un archivo CSV comprimido con gzip desde una URL y lo carga en un DataFrame de Pandas.

    :param url: String con la URL del archivo .gz.
    :param sep: String delimitador para separar campos en el archivo CSV.
    :param header: Parámetro para determinar la fila que debe usarse como nombre de las columnas.
    :return: DataFrame de Pandas con los datos del archivo CSV.
    """
    # Realizar la petición HTTP para obtener el contenido del archivo
    response = requests.get(url)

    # Asegurarse de que la petición fue exitosa
    if response.status_code == 200:
        # Descomprimir el contenido en memoria y leer directamente en un DataFrame
        with io.BytesIO(response.content) as bytes_io:
            with gzip.open(bytes_io, 'rt') as read_file:
                df = pd.read_csv(read_file, sep=sep, header=header)
        return df
    else:
        raise IOError(f"Error al descargar el archivo: {response.status_code}")

def read_csv_from_url(url, sep='\t', header='infer'):
    response = requests.get(url)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep=sep, header=header)
        return df
    else:
        raise IOError(f"Error al descargar el archivo: {response.status_code}")

def plotear_grafico(df):
    # Convertir el 'periodo' a formato de fecha para mejor visualización en el gráfico
    df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')

    # Crear nuevamente el gráfico de línea con Seaborn
    plt.figure(figsize=(14, 7))
    line_plot = sns.lineplot(data=df, x='periodo', y='tn', marker='o', color='blue', linewidth=2.5)

    # Títulos y etiquetas
    line_plot.set_title('Evolución de Toneladas a lo largo del Tiempo', fontsize=16)
    line_plot.set_xlabel('Periodo', fontsize=14)
    line_plot.set_ylabel('Toneladas (tn)', fontsize=14)

    # Establecer el formato del eje x para mostrar todos los periodos
    line_plot.xaxis.set_major_locator(plt.MaxNLocator(len(df['periodo'])))
    line_plot.set_xticklabels(df['periodo'].dt.strftime('%Y-%m'), rotation=45, ha='right')

    # Mostrar el gráfico
    plt.tight_layout()
    plt.show()

def completar_periodos(df, fecha_inicio="2017-01-01", fecha_fin="2020-02-01"):
    """
    Devuelve un DataFrame con todas las combinaciones product_id × mes
    entre fecha_inicio y fecha_fin. Los campos que varían mes a mes se
    dejan en NaN (el modelo los completará después).
    """
    # 1) calendario mensual (primer día de mes)
    meses = pd.date_range(start=fecha_inicio, end=fecha_fin, freq="MS")

    # 2) producto cartesiano product_id × meses
    full_idx = pd.MultiIndex.from_product(
        [df["product_id"].unique(), meses],
        names=["product_id", "periodo"]
    )
    df_full = (df
               .set_index(["product_id", "periodo"])
               .reindex(full_idx)
               .reset_index())

    # 3) columnas estáticas que SÍ podemos rellenar
    cols_static = ["plan_precios_cuidados", "cat1", "cat2", "cat3", "brand", "sku_size"]
    df_full.sort_values(["product_id", "periodo"], inplace=True)
    df_full[cols_static] = (
        df_full.groupby("product_id")[cols_static]
               .ffill()
               .bfill()
    )

    # tn, cust_request_tn y futuros lags quedan en NaN
    return df_full



def generar_delta_lags_optimizado(df, prefijo_lag='lag_', prefijo_delta_lag='delta_lag_'):
    """
    Genera delta lags de manera más eficiente para una columna específica de un DataFrame de pandas.

    :param df: DataFrame original que ya contiene las columnas de lags.
    :param prefijo_lag: Prefijo utilizado para las columnas de lags en el DataFrame.
    :param prefijo_delta_lag: Prefijo utilizado para las nuevas columnas de delta lags.
    :return: DataFrame con las columnas de delta lags añadidas.
    """

    # Identificar todas las columnas de lags en el DataFrame
    columnas_lags = [col for col in df.columns if col.startswith(prefijo_lag)]

    # Generar las columnas de delta lags
    for i in range(len(columnas_lags) - 1):
        col_lag_actual = columnas_lags[i]
        col_lag_siguiente = columnas_lags[i + 1]
        df[prefijo_delta_lag + str(i + 1)] = df[col_lag_actual] - df[col_lag_siguiente]

    return df

# Re-creating the optimized lag generation function after the reset
def generar_lags_optimizado(df, nombre_col_original='tn', cant_lags=12):
    """
    Genera lags de manera más eficiente para una columna específica de un DataFrame de pandas.

    :param df: DataFrame original.
    :param nombre_col_original: Nombre de la columna sobre la cual se quieren calcular los lags.
    :param cant_lags: Cantidad de lags a generar.
    :return: DataFrame con las columnas de lags añadidas.
    """

    # Obtener una lista de productos únicos
    productos_unicos = df['product_id'].unique()

    # Preparar un contenedor para los DataFrames de lags
    lags_list = []

    # Iterar sobre cada producto único
    for producto in productos_unicos:
        # Filtrar el DataFrame por producto
        df_producto = df[df['product_id'] == producto].copy()

        # Generar lags para la columna deseada
        for lag in range(1, cant_lags + 1):
            df_producto[f'lag_{lag}'] = df_producto[nombre_col_original].shift(lag)

        # Añadir el DataFrame de lags a la lista
        lags_list.append(df_producto)

    # Concatenar todos los DataFrames de lags
    df_con_lags = pd.concat(lags_list).sort_index()

    return df_con_lags


def generar_lags_optimizadov2(df, col='tn', n_lags=12, prefijo='lag'):
    """
    Añade columnas lag_1 … lag_n_lags para `col`, por product_id,
    de forma vectorizada (sin bucles externos costosos).

    Parameters
    ----------
    df : pandas.DataFrame
    col : str
        Nombre de la columna sobre la que quieres los lags.
    n_lags : int
        Número de lags a generar.
    prefijo : str
        Prefijo de las nuevas columnas (lag, stock_t, etc.).

    Returns
    -------
    pandas.DataFrame
        El DataFrame original + columnas de lags.
    """
    out = df.copy()
    for k in range(1, n_lags + 1):
        out[f"{prefijo}_{k}"] = (
            out.groupby("product_id")[col].transform(lambda s: s.shift(k))
        )
    return out



def generar_medias_optimizado(df, nombre_col_original='tn', lista_medias_target=[2,3,4,6,10], prefijo_col_media='media'):
    """
    Genera las medias móviles de los últimos N meses de manera más eficiente para una columna específica de un DataFrame de pandas.

    :param df: DataFrame original.
    :param nombre_col_original: Nombre de la columna sobre la cual se quieren calcular las medias móviles.
    :param lista_medias_target: Lista de enteros que representan el número de meses para calcular la media móvil.
    :param prefijo_col_media: Prefijo para las nuevas columnas de medias móviles.
    :return: DataFrame con las columnas de medias móviles añadidas.
    """

    # Obtener una lista de productos únicos
    productos_unicos = df['product_id'].unique()

    # Preparar un contenedor para los DataFrames de medias móviles
    medias_list = []

    # Iterar sobre cada producto único
    for producto in productos_unicos:
        # Filtrar el DataFrame por producto
        df_producto = df[df['product_id'] == producto].copy()

        # Generar medias móviles para cada ventana especificada en lista_medias_target
        for ventana in lista_medias_target:
            col_media = f'{nombre_col_original}_{prefijo_col_media}_{ventana}'
            df_producto[col_media] = df_producto[nombre_col_original].rolling(window=ventana).mean().shift()

        # Añadir el DataFrame de medias móviles a la lista
        medias_list.append(df_producto)

    # Concatenar todos los DataFrames de medias móviles
    df_con_medias = pd.concat(medias_list).sort_index()

    return df_con_medias


class MultiColumnLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoders = {}

    def fit(self, X, y=None):
        for column in X.columns:
            self.encoders[column] = LabelEncoder().fit(X[column])
        return self

    def transform(self, X):
        X_encoded = X.copy()
        for column in X.columns:
            X_encoded[column] = self.encoders[column].transform(X[column])
        return X_encoded

    def inverse_transform(self, X):
        X_decoded = X.copy()
        for column in X.columns:
            X_decoded[column] = self.encoders[column].inverse_transform(X[column])
        return X_decoded


# ────────────────────────────────────────────────────────────────
# 1️⃣  LAGS  (prefijo inteligente)
# ────────────────────────────────────────────────────────────────
def generar_lags_optimizadov2(df, col='tn', n_lags=12, prefijo='lag'):
    """
    Crea columnas lag_k para `col` por product_id.
    ✔ No añade subrayado duplicado:
        prefijo = 'lag'      → lag_1
        prefijo = 'stock_t'  → stock_t1
    """
    sep = "_" if not prefijo.endswith(("_", "t")) else ""   # evita stock_t_1
    out = df.copy()
    for k in range(1, n_lags + 1):
        out[f"{prefijo}{sep}{k}"] = (
            out.groupby("product_id")[col].transform(lambda s: s.shift(k))
        )
    return out

# ────────────────────────────────────────────────────────────────
# 2️⃣  DELTA-LAGS  (regex robusto)
# ────────────────────────────────────────────────────────────────
def generar_delta_lags_optimizado_v2(df, base_prefijo="lag_", prefijo_out="delta_lag_"):
    """
    Delta_k = lag_k − lag_{k+1}  (funciona con 'lag_1' o 'stock_t1').
    """
    out = df.copy()
    cols = sorted(
        [c for c in out.columns if c.startswith(base_prefijo)],
        key=lambda x: int(re.search(r'(\d+)$', x).group(1))
    )
    for i in range(len(cols) - 1):
        k = int(re.search(r'(\d+)$', cols[i]).group(1))
        out[f"{prefijo_out}{k}"] = out[cols[i]] - out[cols[i + 1]]
    return out

# ────────────────────────────────────────────────────────────────
# 3️⃣  MEDIAS  (vectorizado)
# ────────────────────────────────────────────────────────────────
def generar_medias_optimizado_v2(df, col, ventanas=(2,3,4,6,10), prefijo="media"):
    """
    media_k = mean(col, window=k) desplazado 1 periodo.
    """
    out = df.copy()
    g = out.groupby("product_id")[col]
    for w in ventanas:
        out[f"{col}_{prefijo}_{w}"] = (
            g.rolling(window=w, min_periods=1).mean()
              .shift()
              .reset_index(level=0, drop=True)
        )
    return out

# ──────────────────────────────────────────────────────────────
# Función utilitaria
# ──────────────────────────────────────────────────────────────
def eval_producto(
    model,
    df_test,                # DataFrame de diciembre
    product_ids,            # int o lista de ints
    id_col="product_id",
    target_col="tn",        # en log1p
    num_iter=None
):
    """
    Devuelve DataFrame con métricas MAE / MAPE para uno o varios productos.
    """
    if isinstance(product_ids, int):
        product_ids = [product_ids]

    # 1) prepara X_test y predicciones en log
    X_test = prep_strict(df_test)
    y_pred_log = model.predict(X_test, num_iteration=num_iter)

    tmp = df_test.copy()
    tmp["y_pred_log"] = y_pred_log

    # 2) filtra productos seleccionados
    mask = tmp[id_col].isin(product_ids)
    pe   = tmp[mask].copy()

    # 3) pasa a escala real
    pe["y_true_lin"] = np.expm1(pe[target_col])
    pe["y_pred_lin"] = np.expm1(pe["y_pred_log"])
    pe["abs_err"] = np.abs(pe["y_true_lin"] - pe["y_pred_lin"])
    pe["ape"] = np.where(
        pe["y_true_lin"] != 0,
        pe["abs_err"] / pe["y_true_lin"],
        np.nan
    )

    # 4) métricas agregadas por producto
    resumen = (
        pe.groupby(id_col)
          .agg(
              MAE_lin  = ("abs_err", "mean"),
              MAPE_lin = ("ape",    "mean"),
              N_obs    = ("abs_err","size")
          )
          .reset_index()
    )

    return resumen, pe



# ------------------------------------------------------------------
# 3) ENTRENAMIENTO  (con pesos WMAPE)
# ------------------------------------------------------------------
def train_lgb(X_train, y_train, X_valid, y_valid, params,
              num_rounds=3000, early_stop=100, name='model'):

    # Pesos = toneladas reales  (aproxima WMAPE)
    w_train = np.expm1(y_train) if USE_LOG_TARGET else y_train
    w_valid = np.expm1(y_valid) if USE_LOG_TARGET else y_valid

    dtrain = lgb.Dataset(X_train, y_train,
                         weight=w_train,
                         categorical_feature=cat_feats)
    dvalid = lgb.Dataset(X_valid, y_valid,
                         weight=w_valid,
                         categorical_feature=cat_feats)

    booster = lgb.train(
        params,
        dtrain,
        num_boost_round=num_rounds,
        valid_sets=[dvalid],
        valid_names=['valid'],
        callbacks=[
            lgb.early_stopping(early_stop, verbose=False),
            lgb.log_evaluation(200)
        ]
    )
    mae_val = mean_absolute_error(
        y_valid, booster.predict(X_valid, num_iteration=booster.best_iteration)
    )
    print(f"▶ {name} | MAE valid (log): {mae_val:.4f} | best_iter: {booster.best_iteration}")
    return booster

# ------------------------------------------------------------------
# Métrica personalizada
# ------------------------------------------------------------------
def mape_safe(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-8) -> float:
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / (y_true[mask] + eps)))

def wmape(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-8) -> float:
    return np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + eps)


# Métrica WMAPE para LightGBM
def lgb_wmape_eval(y_pred, dataset):
    # y_pred y y_train están en log → pasamos a toneladas
    y_pred_tn = np.expm1(y_pred) if USE_LOG_TARGET else y_pred
    y_true_tn = np.expm1(dataset.get_label()) if USE_LOG_TARGET else dataset.get_label()

    # si usas pesos = tn reales, tómalos de ahí; si no, usa y_true_tn
    weight = dataset.get_weight()
    if weight is None:
        weight = y_true_tn

    wmape = np.sum(np.abs(y_true_tn - y_pred_tn)) / (np.sum(weight) + 1e-8)
    return "WMAPE", wmape, False        # False → cuanto más bajo, mejor

def feature_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    """
    Recibe un df que YA incluye stock_t0,
    y devuelve el df con todas las features.
    """
    df = df.copy()

    # 1) Calendar
    df["periodo"] = pd.to_datetime(df["periodo"]) # Ensure periodo is datetime
    df["month"]     = df["periodo"].dt.month.astype("int8")
    df["quarter"]   = df["periodo"].dt.quarter.astype("int8")
    df["month_sin"] = np.sin(2*np.pi*df["month"]/12)
    df["month_cos"] = np.cos(2*np.pi*df["month"]/12)

    # 2) Asegura stock_t0 (si no está, lo crea con NaN)
    if "stock_t0" not in df.columns:
        df["stock_t0"] = np.nan

    # 3) Lags de tn (1–24)
    df = generar_lags_optimizadov2(df, col="tn", n_lags=24)

    # 4) Deltas y medias sobre tn
    df = generar_delta_lags_optimizado_v2(df)
    df = generar_medias_optimizado_v2(df, col="tn", ventanas=(2,3,4,6,10))

    # 5) Features de stock
    # Ensure 'lag_1' is created before accessing it
    df["flag_stock_disp"] = (~df["stock_t0"].isna()).astype("int8")
    df["ratio_tn_stock"]  = (
        df["lag_1"] / (df["stock_t0"] + 1e-3) # Access 'lag_1' after it's generated
    ).where(df["flag_stock_disp"] == 1, np.nan)

    # 6) YoY (lag 12, delta_yoy, ratio_yoy)
    df = df.sort_values(["product_id","periodo"])

    # Ensure 'lag_1' and 'lag_13' are created before accessing them
    df["delta_yoy"] = df["lag_1"] - df["lag_13"] # Access 'lag_1' and 'lag_13' after they are generated
    df["ratio_yoy"] = df["lag_1"] / (df["lag_13"] + 1e-6) # Access 'lag_1' and 'lag_13' after they are generated

    return df

In [4]:
# URL del archivo .gz
url_tb_sellout_02 = 'https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/sell-in.txt.gz'
url_tb_stocks_02 = 'https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_stocks.txt'
url_tb_productos_02 = 'https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/tb_productos.txt'





# Uso de la función
df_tb_sellout_02 = cargar_dataset(url_tb_sellout_02)
df_tb_stocks_02= read_csv_from_url(url_tb_stocks_02)
df_tb_productos_02= read_csv_from_url(url_tb_productos_02)


In [5]:
df_tb_sellout_02.head(50)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452
5,201701,10080,20524,0,1,0.01514,0.01514
6,201701,10015,20524,0,4,0.10600,0.10600
7,201701,10062,20524,0,1,0.18928,0.18928
8,201701,10159,20524,0,3,0.02271,0.02271
9,201701,10183,20524,0,1,0.01514,0.01514


In [6]:
productospredecir=pd.read_csv("/content/sample_data/product_id_apredecir201912.txt",sep="\t",header=0)

In [7]:
df_tb_sellout_02['periodo'].unique()

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912])

In [8]:
# Juntamos los datasets Sellout con el maestro de productos
df_sellout_with_cat = pd.merge(df_tb_sellout_02, df_tb_productos_02[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']], on='product_id', how='left')
# Agrupamos por categorías de productos
df_aggregated = df_sellout_with_cat.groupby(['periodo', 'product_id', 'plan_precios_cuidados',
                                             'cat1', 'cat2', 'cat3', 'brand', 'sku_size']).agg({'cust_request_tn': 'sum','tn': 'sum'}).reset_index()
# 1) Asegúrate de que ambas columnas product_id sean del mismo tipo
df_aggregated["product_id"]      = df_aggregated["product_id"].astype(int)
productospredecir["product_id"] = productospredecir["product_id"].astype(int)

# 2) El join: basta con quedarnos con los IDs que aparecen en la lista
df_filtrado = (
    df_aggregated.merge(
        productospredecir[["product_id"]].drop_duplicates(),  # solo la llave
        on="product_id",
        how="inner"                                           # ← inner join
    )
)

print("Filas antes :", len(df_aggregated))
print("Filas después:", len(df_filtrado))

df = df_filtrado.copy()                        # tu tabla de origen
df["periodo"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")
df_full=df

Filas antes : 31014
Filas después: 22349


### Feature engineering - se transforma a log y se hace primero los lags y deltas sobre las variables crudas como vienen

**Armamos un product id ficticio para correr las escalas ahora lo ignoramos**

In [9]:

df_subset=df_full

# Crear un nuevo DataFrame con los mismos periodos que df_subset
periodos = df_subset['periodo'].unique()  # Extraer los periodos únicos
datos_ficticios = {
    'periodo': periodos,
    'product_id': 99999,  # ID del producto ficticio
    'plan_precios_cuidados': 0.0,
    'cat1': 'HC',
    'cat2': 'Artificial',
    'cat3': 'Artificial',
    'brand': 'Producto_artificial',
    'sku_size': 3000.0,
    # Inicializamos cust_request_tn y tn con ceros, los actualizaremos luego
    'cust_request_tn': [0] * len(periodos),
    'tn': [0] * len(periodos)
}

df_ficticio = pd.DataFrame(datos_ficticios)

# Ahora, actualizamos 'cust_request_tn' y 'tn' con 10 veces los valores del producto 20001
# Asumimos que 'product_id' es único por periodo en df_subset
for periodo in periodos:
    # Encuentra los valores para el producto 20001 en el periodo actual
    valores_producto = df_subset[(df_subset['periodo'] == periodo) & (df_subset['product_id'] == 20001)]
    if not valores_producto.empty:
        df_ficticio.loc[df_ficticio['periodo'] == periodo, 'cust_request_tn'] = valores_producto['cust_request_tn'].values[0] * 3
        df_ficticio.loc[df_ficticio['periodo'] == periodo, 'tn'] = valores_producto['tn'].values[0] * 3

# Concatenar el DataFrame ficticio con el original
df_subset_con_ficticio = pd.concat([df_subset, df_ficticio], ignore_index=True)

/tmp/ipython-input-9-1343700785.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2813.18151' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_ficticio.loc[df_ficticio['periodo'] == periodo, 'cust_request_tn'] = valores_producto['cust_request_tn'].values[0] * 3
/tmp/ipython-input-9-1343700785.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2804.31666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_ficticio.loc[df_ficticio['periodo'] == periodo, 'tn'] = valores_producto['tn'].values[0] * 3


***aplicamos Lags a todo , stocks - tn - y le hacemos tambien promedios y delta lags, lo que paso es que cuando meti el stock , mejoro pero despues empeor, pueden ser tres cos

In [10]:
df_final=df_full

# ───────────────────────────────────────────────────────────
# 🔖 FEATURE: Antigüedad en meses desde primera aparición
# ───────────────────────────────────────────────────────────
# Primero: calculamos la fecha mínima (primera aparición)
fecha_inicio = df_final.groupby('product_id')['periodo'].min().reset_index()
fecha_inicio.columns = ['product_id', 'fecha_primera_aparicion']

# Añadimos esta fecha original al DataFrame principal
df_final = df_final.merge(fecha_inicio, on='product_id', how='left')

# Creamos la variable "antiguedad_meses"
df_final['antiguedad_meses'] = (
    (df_final['periodo'].dt.year - df_final['fecha_primera_aparicion'].dt.year) * 12 +
    (df_final['periodo'].dt.month - df_final['fecha_primera_aparicion'].dt.month)
).astype(int)

# Verificamos rápido
print(df_final[['product_id', 'periodo', 'fecha_primera_aparicion', 'antiguedad_meses']].head(10))


   product_id    periodo fecha_primera_aparicion  antiguedad_meses
0       20001 2017-01-01              2017-01-01                 0
1       20002 2017-01-01              2017-01-01                 0
2       20003 2017-01-01              2017-01-01                 0
3       20004 2017-01-01              2017-01-01                 0
4       20005 2017-01-01              2017-01-01                 0
5       20006 2017-01-01              2017-01-01                 0
6       20007 2017-01-01              2017-01-01                 0
7       20008 2017-01-01              2017-01-01                 0
8       20009 2017-01-01              2017-01-01                 0
9       20010 2017-01-01              2017-01-01                 0


In [11]:
df_final.drop(columns=['fecha_primera_aparicion'], inplace=True)

In [12]:
df_final=df_final


# ------------------------------------------------------------------
# 0)  BASE: ventas lineales
# ------------------------------------------------------------------
feats = df_final.copy()
feats["periodo"] = pd.to_datetime(feats["periodo"], format="%Y%m")

# ------------------------------------------------------------------
# 1) MERGE con stock desplazado +1 mes (stock_t0)
# ------------------------------------------------------------------
stock = (
    df_tb_stocks_02.copy()
        .assign(periodo=lambda d:
                pd.to_datetime(d["periodo"].astype(str), format="%Y%m")
                + pd.offsets.MonthEnd(0)      # 201810 → 2018-10-31
                + pd.offsets.MonthBegin(1))   #        → 2018-11-01
        .rename(columns={"stock_final": "stock_t0"})
)

featsandStock = feats.merge(
    stock[["product_id", "periodo", "stock_t0"]],
    on=["product_id", "periodo"],
    how="left"
)


In [13]:
featsandStock.columns

Index(['periodo', 'product_id', 'plan_precios_cuidados', 'cat1', 'cat2',
       'cat3', 'brand', 'sku_size', 'cust_request_tn', 'tn',
       'antiguedad_meses', 'stock_t0'],
      dtype='object')

In [14]:
# ────────────────────────────────────────────────────
# 1) Generas todas las features de una sola vez
# ────────────────────────────────────────────────────
featspipelined = feature_pipeline(featsandStock)   # raw_df incluye periodo, product_id, tn, etc.

# ────────────────────────────────────────────────────
# 2) Target y sample weights
# ────────────────────────────────────────────────────
featspipelined["tn_log"] = np.log1p(featspipelined["tn"])
y = featspipelined["tn_log"]
weights = np.expm1(y)

# ────────────────────────────────────────────────────
# 3) Filtrado histórico de entrenamiento
# ────────────────────────────────────────────────────
mask = featspipelined["periodo"].between("2017-01-01","2019-12-01")
readyforModel = featspipelined.loc[mask].copy()




In [15]:
readyforModel.tail()


,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,...,tn_media_2,tn_media_3,tn_media_4,tn_media_6,tn_media_10,flag_stock_disp,ratio_tn_stock,delta_yoy,ratio_yoy,tn_log
19228,2019-08-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.01265,0.01265,...,0.021545,0.045307,0.059413,0.072028,0.072028,1,0.001729,NaN,NaN,0.012571
20008,2019-09-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.01856,0.01856,...,0.007440,0.018580,0.037143,0.062132,0.062132,1,0.011650,NaN,NaN,0.018390
20788,2019-10-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.02079,0.02079,...,0.015605,0.011147,0.018575,0.044810,0.055907,1,0.021158,NaN,NaN,0.020577
21568,2019-11-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.03341,0.03341,...,0.019675,0.017333,0.013558,0.031320,0.051518,1,0.019615,NaN,NaN,0.032864
22348,2019-12-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.00892,0.00892,...,0.027100,0.024253,0.021352,0.021417,0.049506,1,0.031456,NaN,NaN,0.008880


***Modelo listo para correr***

***Vista Resumida***

In [16]:
# --- Info y resumen de readyforModel -----------------------------------
import pandas as pd
import numpy as np
import io, sys

# A) info() en texto
buf = io.StringIO()
readyforModel.info(buf=buf)
print(">>>  readyforModel.info()")
print(buf.getvalue())

# B) NAs por columna, ordenados
print("\n>>>  Top 30 columnas con más NAs")
na = readyforModel.isna().sum().sort_values(ascending=False)
display(na.head(30))

# C) Estadísticos básicos de las 15 primeras numéricas
print("\n>>>  describe() primeras numéricas")
display(readyforModel.select_dtypes(include=[np.number]).iloc[:, :15].describe().T)

# D) Contar cuántas columnas usan tn_t (terminan en '_t0')
cols_t0 = [c for c in readyforModel.columns if '_t' in c and c.endswith('0')]
print(f"\n>>>  Columnas basadas en tn_t0: {len(cols_t0)}")
print(cols_t0[:15])   # muestra las primeras 15

>>>  readyforModel.info()
<class 'pandas.core.frame.DataFrame'>
Index: 22349 entries, 0 to 22348
Data columns (total 73 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   periodo                22349 non-null  datetime64[ns]
 1   product_id             22349 non-null  int64         
 2   plan_precios_cuidados  22349 non-null  int64         
 3   cat1                   22349 non-null  object        
 4   cat2                   22349 non-null  object        
 5   cat3                   22349 non-null  object        
 6   brand                  22349 non-null  object        
 7   sku_size               22349 non-null  float64       
 8   cust_request_tn        22349 non-null  float64       
 9   tn                     22349 non-null  float64       
 10  antiguedad_meses       22349 non-null  int64         
 11  stock_t0               9941 non-null   float64       
 12  month                  22349 non-null  

,0
lag_24,16087
delta_lag_23,16087
delta_lag_22,15519
lag_23,15519
lag_22,14950
delta_lag_21,14950
delta_lag_20,14375
lag_21,14375
lag_20,13788
delta_lag_19,13788



>>>  describe() primeras numéricas


,count,mean,std,min,25%,50%,75%,max
product_id,22349.0,20479.013423,334.475371,20001.00000,20200.000000,2.041100e+04,20730.000000,21276.00000
plan_precios_cuidados,22349.0,0.010649,0.102647,0.00000,0.000000,0.000000e+00,0.000000,1.00000
sku_size,22349.0,447.639134,831.235708,1.00000,89.000000,2.200000e+02,475.000000,10000.00000
cust_request_tn,22349.0,51.364013,129.425314,0.00089,3.010850,1.193662e+01,36.272670,2423.70881
tn,22349.0,50.230669,125.016406,0.00089,2.991610,1.190046e+01,35.983630,2295.19832
antiguedad_meses,22349.0,15.974630,10.374556,0.00000,7.000000,1.500000e+01,25.000000,35.00000
stock_t0,9941.0,21.577077,60.656471,-13.66656,1.476380,6.240780e+00,19.575660,1562.02448
month,22349.0,6.682760,3.445702,1.00000,4.000000,7.000000e+00,10.000000,12.00000
quarter,22349.0,2.558414,1.116071,1.00000,2.000000,3.000000e+00,4.000000,4.00000
month_sin,22349.0,-0.030315,0.703918,-1.00000,-0.866025,-2.449294e-16,0.500000,1.00000



>>>  Columnas basadas en tn_t0: 1
['stock_t0']


# Armamos Multiples Modelos con diferentes complejidades

## Modelo 1 lightGbm clasico - Sin search de hiperparametros

In [17]:
readyforModel.columns

Index(['periodo', 'product_id', 'plan_precios_cuidados', 'cat1', 'cat2',
       'cat3', 'brand', 'sku_size', 'cust_request_tn', 'tn',
       'antiguedad_meses', 'stock_t0', 'month', 'quarter', 'month_sin',
       'month_cos', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6',
       'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13',
       'lag_14', 'lag_15', 'lag_16', 'lag_17', 'lag_18', 'lag_19', 'lag_20',
       'lag_21', 'lag_22', 'lag_23', 'lag_24', 'delta_lag_1', 'delta_lag_2',
       'delta_lag_3', 'delta_lag_4', 'delta_lag_5', 'delta_lag_6',
       'delta_lag_7', 'delta_lag_8', 'delta_lag_9', 'delta_lag_10',
       'delta_lag_11', 'delta_lag_12', 'delta_lag_13', 'delta_lag_14',
       'delta_lag_15', 'delta_lag_16', 'delta_lag_17', 'delta_lag_18',
       'delta_lag_19', 'delta_lag_20', 'delta_lag_21', 'delta_lag_22',
       'delta_lag_23', 'tn_media_2', 'tn_media_3', 'tn_media_4', 'tn_media_6',
       'tn_media_10', 'flag_stock_disp', 'ratio_tn_stock', 'delta

Modelo 1- clasico

In [18]:
DROP_COLS_NOW = [
    'tn',       # target
    'tn_log',   # model target
    'periodo',  # <— must drop the datetime
    'cust_request_tn'
    #'ratio_tn_stock',
    #'flag_stock_disp',

    #'stock_t0'
]
cat_feats = ['cat1','cat2','cat3','brand','month','quarter']

Empezamos a correr el modelo

Modelo 1-LightGbm base

In [19]:

# ------------------------------------------------------------------
# CONFIGURACIÓN
# ------------------------------------------------------------------
import pandas as pd, numpy as np, lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# ------------------------------------------------------------------
# CONFIGURACIÓN
# ------------------------------------------------------------------

DROP_COLS_NOW = [
    'tn',       # target
    'tn_log',   # model target
    'periodo',  # <— must drop the datetime
    'cust_request_tn'
    #'ratio_tn_stock',
    #'flag_stock_disp',

    #'stock_t0'
]
cat_feats = ['cat1','cat2','cat3','brand','month','quarter']


# ------------------------------------------------------------------
# 1) SPLITS
# ------------------------------------------------------------------
train_oct = readyforModel[readyforModel['periodo'] <= '2019-10-01'].copy()
valid_nov = readyforModel[readyforModel['periodo'] == '2019-11-01'].copy()
test_dec  = readyforModel[readyforModel['periodo'] == '2019-12-01'].copy()

# ------------------------------------------------------------------
# 2) PREPROCESAMIENTO SIN ‘LEAKAGE’
# ------------------------------------------------------------------
# ------------------------------------------------------------------
# 2) PREPROCESAMIENTO SIN ‘LEAKAGE’
# ------------------------------------------------------------------
def prep_strict(df: pd.DataFrame) -> pd.DataFrame:
    """
    Elimina columnas de fuga + cualquier datetime + tipa categóricas.
    """
    # 1) Quitamos las columnas indicadas
    X = df.drop(columns=DROP_COLS_NOW, errors="ignore").copy()

    # 2) Además, eliminamos cualquier columna datetime que quedase
    dt_cols = X.select_dtypes(include=['datetime64[ns]']).columns.tolist()
    if dt_cols:
        X.drop(columns=dt_cols, inplace=True)

    # 3) Tipeamos las categóricas
    for c in cat_feats:
        if c in X.columns:
            X[c] = X[c].astype('category')
    return X

USE_LOG_TARGET = True                 # ← pon False si prefieres toneladas
TARGET_COL     = 'tn_log' if USE_LOG_TARGET else 'tn'

X_tr   = prep_strict(train_oct)
y_tr   = train_oct[TARGET_COL]

X_val  = prep_strict(valid_nov)
y_val  = valid_nov[TARGET_COL]

X_test = prep_strict(test_dec)
y_true_tn = test_dec['tn']            # tn reales para métricas finales


# ------------------------------------------------------------------
# 4) MODELOS
# ------------------------------------------------------------------
seed = 42
params_base = {
    "objective": "regression",
    "metric": "mae",
    "learning_rate": 0.1,
    "num_leaves": 50,
    "seed": seed,
    "feature_fraction_seed": seed,
    "bagging_seed": seed,
    "nthread": 1,
    "deterministic": True
}

params_tuned = {
    "objective": "regression",
    "metric": "mae",
    "learning_rate": 0.08813259224734658,
    "num_leaves": 79,
    "min_child_samples": 100,
    "feature_fraction": 0.9833497436436693,
    "bagging_fraction": 0.9951622648438583,
    "bagging_freq": 0,
    "lambda_l2": 0.3939212978268534,
    "seed": 42,
    "feature_fraction_seed": 42,
    "bagging_seed": 42,
    "nthread": 1,
    "deterministic": True,
    "verbose": -1
}


base_model = train_lgb(X_tr, y_tr, X_val, y_val,
                       params_base, num_rounds=1000, early_stop=50, name='BASE')

tuned_model = train_lgb(X_tr, y_tr, X_val, y_val,
                        params_tuned, num_rounds=4000, early_stop=200, name='TUNED')

def wmape(y_true, y_pred, eps=1e-8):
    return np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + eps)


# ------------------------------------------------------------------
# 5) EVALUACIÓN EN DIC-19  (toneladas reales)
# ------------------------------------------------------------------
for tag, model in [("BASE", base_model), ("TUNED", tuned_model)]:
    y_pred_log = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred_tn  = np.expm1(y_pred_log) if USE_LOG_TARGET else y_pred_log

    mae   = mean_absolute_error(y_true_tn, y_pred_tn)
    mape  = mape_safe(y_true_tn, y_pred_tn)
    wmAPE = wmape(y_true_tn, y_pred_tn)

    print(f"{tag:>5} | MAE: {mae:.2f} | MAPE: {mape:.2%} | WMAPE: {wmAPE:.2%}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023938 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14818
[LightGBM] [Info] Number of data points in the train set: 20789, number of used features: 69
[LightGBM] [Info] Start training from score 5.214541
▶ BASE | MAE valid (log): 0.2503 | best_iter: 89
[200]	valid's l1: 0.247524
▶ TUNED | MAE valid (log): 0.2423 | best_iter: 112
 BASE | MAE: 7.51 | MAPE: 196.71% | WMAPE: 23.31%
TUNED | MAE: 7.86 | MAPE: 205.05% | WMAPE: 24.39%


modelo con tweedie


LightGbm Tweedie busqueda de Powers

In [20]:
# ------------------------------------------------------------------
# Loop rápido de p Tweedie y captura de resultados
# ------------------------------------------------------------------
powers = [1.1, 1.2, 1.3, 1.4, 1.5]
results = []

for p in powers:
    params_p = params_base.copy()
    params_p.update({
        "objective": "tweedie",
        "tweedie_variance_power": p,
    })
    model_p = train_lgb(
        X_tr, y_tr,
        X_val, y_val,
        params_p,
        num_rounds=4000,
        early_stop=200,
        name=f"TWEED_p{p}"
    )

    # --- eval en dic-19 ---
    y_pred = np.expm1(model_p.predict(X_test, num_iteration=model_p.best_iteration))
    wm    = wmape(y_true_tn, y_pred)
    results.append((p, wm))
    print(f"p={p:.1f} | WMAPE: {wm:.2%}")

# ------------------------------------------------------------------
# Extraer el mejor p
# ------------------------------------------------------------------
best_p, best_wmape = min(results, key=lambda x: x[1])
print(f"\nMejor p = {best_p:.1f} con WMAPE = {best_wmape:.2%}")

# ------------------------------------------------------------------
# 1) Definimos hiper-parámetros usando best_p
# ------------------------------------------------------------------
params_tweed_opt = params_base.copy()
params_tweed_opt.update({
    "objective":              "tweedie",
    "tweedie_variance_power": best_p,
    # Opcionalmente afina learning_rate, num_leaves, etc.
})

# ------------------------------------------------------------------
# 2) Entrenamos el modelo final con el best_p
# ------------------------------------------------------------------
tweed_opt_model = train_lgb(
    X_tr, y_tr,
    X_val, y_val,
    params_tweed_opt,
    num_rounds=4000,
    early_stop=300,
    name=f"TWEED_p{best_p:.1f}"
)

# ------------------------------------------------------------------
# 3) Evaluación comparativa en DIC-19
# ------------------------------------------------------------------
for tag, model in [("BASE",    base_model),
                   ("TUNED",   tuned_model),
                   (f"TWEED_{best_p:.1f}", tweed_opt_model)]:
    y_pred_log = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred_tn  = np.expm1(y_pred_log) if USE_LOG_TARGET else y_pred_log

    mae   = mean_absolute_error(y_true_tn, y_pred_tn)
    mape  = mape_safe(y_true_tn, y_pred_tn)
    wmAPE = wmape(y_true_tn, y_pred_tn)

    print(f"{tag:>10} | MAE: {mae:.2f} | MAPE: {mape:.2%} | WMAPE: {wmAPE:.2%}")


[200]	valid's l1: 0.267629
▶ TWEED_p1.1 | MAE valid (log): 0.2577 | best_iter: 55
p=1.1 | WMAPE: 24.26%
[200]	valid's l1: 0.251076
▶ TWEED_p1.2 | MAE valid (log): 0.2539 | best_iter: 53
p=1.2 | WMAPE: 24.14%
[200]	valid's l1: 0.25521
▶ TWEED_p1.3 | MAE valid (log): 0.2510 | best_iter: 53
p=1.3 | WMAPE: 23.93%
[200]	valid's l1: 0.263089
▶ TWEED_p1.4 | MAE valid (log): 0.2556 | best_iter: 47
p=1.4 | WMAPE: 23.42%
[200]	valid's l1: 0.251246
▶ TWEED_p1.5 | MAE valid (log): 0.2628 | best_iter: 35
p=1.5 | WMAPE: 25.83%

Mejor p = 1.4 con WMAPE = 23.42%
[200]	valid's l1: 0.263089
▶ TWEED_p1.4 | MAE valid (log): 0.2556 | best_iter: 47
      BASE | MAE: 7.51 | MAPE: 196.71% | WMAPE: 23.31%
     TUNED | MAE: 7.86 | MAPE: 205.05% | WMAPE: 24.39%
 TWEED_1.4 | MAE: 7.55 | MAPE: 205.85% | WMAPE: 23.42%


comparativo

###busqueda de hiper parametros con optuna tomar la segunda celda la primera es dejaral

este optuna ya tiene los hiperparametros hardcodeados , despues de 2hs de correr otro que encontr lo s mejores

In [21]:
# Redefinimos WMAPE con nombre nuevo para evitar colisiones
def compute_wmape(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-8) -> float:
    return np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + eps)

In [22]:
#este codigo funciona bien


import optuna
import copy
import numpy as np
import pandas as pd
import lightgbm as lgb  # Import lightgbm
from sklearn.metrics import mean_absolute_error

# -------------------------------------------------------------------
# Precondiciones: ya tienes X_tr, y_tr (log1p), X_val, y_val,
#                wmape(y_true, y_pred) y train_lgb(...) definidos,
#                params_base con tu base Tweedie
# -------------------------------------------------------------------

# ╔══════════════════════════════════════════════════════════╗
#   Optuna objective  ▶  búsqueda fina alrededor del best set
# ╚══════════════════════════════════════════════════════════╝
def objective(trial):
    """
    Parte del best-set que encontraste y ajusta +/- un rango razonable.
    Devuelve WMAPE (más bajo = mejor).
    """
    params = params_base.copy() | {
        "objective": "regression",
        "metric":    "mae",
        "seed":      42,
        "deterministic": True,
        # ---------- hp que SÍ ajustaremos ----------
        "tweedie_variance_power": trial.suggest_float("p",
                                   1.05, 1.25, step=0.01),     # alrededor 1.14
        "learning_rate":          trial.suggest_float("lr",
                                   0.015, 0.05, log=True),     # alrededor 0.03
        "num_leaves":             trial.suggest_int("num_leaves",
                                   16, 48, step=4),            # alrededor 23
        "min_child_samples":      trial.suggest_int("min_child_samples",
                                   120, 250, step=10),         # alrededor 188
        "feature_fraction":       trial.suggest_float("feature_fraction",
                                   0.55, 0.85),                # alrededor 0.63
        "bagging_fraction":       trial.suggest_float("bagging_fraction",
                                   0.55, 0.85),                # alrededor 0.66
        "lambda_l2":              trial.suggest_float("lambda_l2",
                                   0.0, 1.0),                  # dejamos libre
    }

    # regla para bagging_freq
    params["bagging_freq"] = 1 if params["bagging_fraction"] < 1.0 else 0

    # ---------- entrenamiento ----------
    booster = train_lgb(
        X_tr, y_tr,
        X_val, y_val,
        params,
        num_rounds=2000,          # menos vueltas: exploración más rápida
        early_stop=200,
        name=f"opt-{trial.number}"
    )

    trial.set_user_attr("best_iteration", booster.best_iteration)

    # ---------- métrica a minimizar ----------
    y_pred = np.expm1(booster.predict(X_val, num_iteration=booster.best_iteration))
    y_true = np.expm1(y_val)
    return compute_wmape(y_true, y_pred)

# ────────── Lanza la búsqueda (40 trials) ──────────
study = optuna.create_study(direction="minimize",
                            sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=65, show_progress_bar=True)

print("■ Mejor WMAPE :", study.best_value)
print("■ Mejores hps :", study.best_params)
print("■ best_iter   :", study.best_trial.user_attrs['best_iteration'])

# -------------------------------------------------------------------
#  5) Re-entrena el modelo final con train+valid (sin mismatch cats)
# -------------------------------------------------------------------

# 5.1) Construye X_trval, y_trval, w_trval
X_trval = pd.concat([X_tr, X_val])
y_trval = pd.concat([y_tr, y_val])
w_trval = np.expm1(y_trval)

# 5.2) Función para codificar categorías
def encode_cats(df, cat_feats):
    df2 = df.copy()
    for c in cat_feats:
        if c in df2.columns:
            df2[c] = pd.Categorical(df2[c]).codes.astype("int32")
    return df2

# 5.3) Numéricos para train y valid
X_trval_num = encode_cats(X_trval, cat_feats)
X_val_num   = encode_cats(X_val,   cat_feats)

# 5.4) Crea los Datasets sin pasar categorical_feature
dtrain_full = lgb.Dataset(
    X_trval_num, y_trval,
    weight=w_trval
)
dvalid = lgb.Dataset(
    X_val_num, y_val,
    weight=np.expm1(y_val)
)

# 5.5) Monta best_params y best_iteration para el re-entrenamiento
best_hyperparams = study.best_params
best_iteration   = study.best_trial.user_attrs["best_iteration"]

# Combina con tu params_base original
best_params = params_base.copy()
best_params.update(best_hyperparams)

# 5.6) Ahora entrena usando early-stopping y tu feval de WMAPE
final_model = lgb.train(
    best_params,
    dtrain_full,
    num_boost_round=best_iteration,       # usa la iteración óptima
    valid_sets=[dtrain_full, dvalid],
    valid_names=["train_full", "valid"],
    callbacks=[
        lgb.log_evaluation(period=200),
        lgb.early_stopping(stopping_rounds=200, verbose=False)
    ],
    feval=lgb_wmape_eval
)

print("✅ final_model re-entrenado sin mismatch de cats")


[I 2025-07-21 00:39:11,107] A new study created in memory with name: no-name-0e3d45a8-fb87-4dbf-917e-9a0e08ea4a90


  0%|          | 0/65 [00:00<?, ?it/s]

[200]	valid's l1: 0.242658
[400]	valid's l1: 0.237583
▶ opt-0 | MAE valid (log): 0.2502 | best_iter: 300
[I 2025-07-21 00:39:18,953] Trial 0 finished with value: 0.21605234424502687 and parameters: {'p': 1.12, 'lr': 0.047119379497769384, 'num_leaves': 40, 'min_child_samples': 200, 'feature_fraction': 0.596805592132731, 'bagging_fraction': 0.5967983561008608, 'lambda_l2': 0.05808361216819946}. Best is trial 0 with value: 0.21605234424502687.
[200]	valid's l1: 0.246562
[400]	valid's l1: 0.242591
▶ opt-1 | MAE valid (log): 0.2702 | best_iter: 362
[I 2025-07-21 00:39:30,427] Trial 1 finished with value: 0.225160311835754 and parameters: {'p': 1.23, 'lr': 0.030931538521705968, 'num_leaves': 40, 'min_child_samples': 120, 'feature_fraction': 0.8409729556485983, 'bagging_fraction': 0.7997327922401265, 'lambda_l2': 0.21233911067827616}. Best is trial 0 with value: 0.21605234424502687.
[200]	valid's l1: 0.256222
[400]	valid's l1: 0.245585
[600]	valid's l1: 0.242313
[800]	valid's l1: 0.242195
▶ o

In [23]:
from pprint import pprint   # sólo para un print bonito

# --- hiperparámetros del booster ---
pprint(final_model.params)


{'bagging_fraction': 0.5967983561008608,
 'bagging_seed': 42,
 'deterministic': True,
 'feature_fraction': 0.596805592132731,
 'feature_fraction_seed': 42,
 'lambda_l2': 0.05808361216819946,
 'learning_rate': 0.1,
 'lr': 0.047119379497769384,
 'metric': 'mae',
 'min_child_samples': 200,
 'nthread': 1,
 'num_iterations': 300,
 'num_leaves': 40,
 'objective': 'regression',
 'p': 1.12,
 'seed': 42}


No correr si no se considera necesario - Optuna 250 iteraciones - no cambia mucho no vale la pena

###Comparativo de modelos y alternativas lightGbm

Modelos a comparar

In [24]:
# ------------------------------------------------------------------
# 3)  Evaluación - aca hay que agruegar el modelo neuvo - ya estan los uqe ya entrenamos
# ------------------------------------------------------------------
model_list = [
    ("BASE",   base_model),
    ("TUNED",  tuned_model),
    ("TWEED",  tweed_opt_model),
    ("FINAL",  final_model)
]


In [25]:
from sklearn.metrics import mean_absolute_error
import numpy as np, pandas as pd

# ------------------------------------------------------------------
# 1)  Capturamos el orden de categorías del entrenamiento
# ------------------------------------------------------------------
cat_levels = {
    c: pd.Categorical(X_trval[c]).categories
    for c in cat_feats
}

def encode_like_train(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c, cats in cat_levels.items():
        if c in out.columns:
            out[c] = (
                pd.Categorical(out[c], categories=cats)
                  .codes.astype("int32")
            )
    return out

# ------------------------------------------------------------------
# 2)  Función WMAPE segura (no será sobrescrita)
# ------------------------------------------------------------------
def wmape_func(y_true, y_pred, eps=1e-8):
    return np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + eps)

# ------------------------------------------------------------------
# 3)  Evaluación
# ------------------------------------------------------------------
model_list = [
    ("BASE",   base_model),
    ("TUNED",  tuned_model),
    ("TWEED",  tweed_opt_model),
    ("FINAL",  final_model)
]

for tag, model in model_list:
    cols   = model.feature_name()
    X_eval = encode_like_train(X_test[cols])
    X_mat  = X_eval.to_numpy(dtype=np.float32)

    y_pred = np.expm1(
        model.predict(X_mat, num_iteration=model.best_iteration or 1)
    )
    mae   = mean_absolute_error(y_true_tn, y_pred)
    mape  = mape_safe(y_true_tn, y_pred)
    wmp   = wmape_func(y_true_tn, y_pred)

    print(f"{tag:>6} | MAE: {mae:.2f} | MAPE: {mape:.2%} | WMAPE: {wmp:.2%}")



  BASE | MAE: 7.51 | MAPE: 196.71% | WMAPE: 23.31%
 TUNED | MAE: 7.86 | MAPE: 205.05% | WMAPE: 24.39%
 TWEED | MAE: 7.55 | MAPE: 205.85% | WMAPE: 23.42%
 FINAL | MAE: 8.50 | MAPE: 149.24% | WMAPE: 26.36%


si queremos agrergar modelos hay que hacerlo aca -- y luego lo nombramos choosen Model en la seccion siguiente  

### Modelo nuevo - Agregar aqui



**Agregamos el modelo nuevo a la lista de abajo, por favor tener en cuenta que si copiamos una celda de arriba hay que cambiar el nombre del output si no sobreescribimos lo que ya hicimos --**

vamos a probar los mismos modelos sin log

In [26]:

# ------------------------------------------------------------------
# CONFIGURACIÓN
# ------------------------------------------------------------------
import pandas as pd, numpy as np, lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# ------------------------------------------------------------------
# CONFIGURACIÓN
# ------------------------------------------------------------------

DROP_COLS_NOW = [
    'tn',       # target
    'tn_log',   # model target
    'periodo',  # <— must drop the datetime
    'cust_request_tn'
    #'ratio_tn_stock',
    #'flag_stock_disp',

    #'stock_t0'
]
cat_feats = ['cat1','cat2','cat3','brand','month','quarter']


# ------------------------------------------------------------------
# 1) SPLITS
# ------------------------------------------------------------------
train_oct = readyforModel[readyforModel['periodo'] <= '2019-10-01'].copy()
valid_nov = readyforModel[readyforModel['periodo'] == '2019-11-01'].copy()
test_dec  = readyforModel[readyforModel['periodo'] == '2019-12-01'].copy()

# ------------------------------------------------------------------
# 2) PREPROCESAMIENTO SIN ‘LEAKAGE’
# ------------------------------------------------------------------
# ------------------------------------------------------------------
# 2) PREPROCESAMIENTO SIN ‘LEAKAGE’
# ------------------------------------------------------------------
def prep_strict(df: pd.DataFrame) -> pd.DataFrame:
    """
    Elimina columnas de fuga + cualquier datetime + tipa categóricas.
    """
    # 1) Quitamos las columnas indicadas
    X = df.drop(columns=DROP_COLS_NOW, errors="ignore").copy()

    # 2) Además, eliminamos cualquier columna datetime que quedase
    dt_cols = X.select_dtypes(include=['datetime64[ns]']).columns.tolist()
    if dt_cols:
        X.drop(columns=dt_cols, inplace=True)

    # 3) Tipeamos las categóricas
    for c in cat_feats:
        if c in X.columns:
            X[c] = X[c].astype('category')
    return X

USE_LOG_TARGET = False                 # ← pon False si prefieres toneladas
TARGET_COL     = 'tn_log' if USE_LOG_TARGET else 'tn'

X_tr   = prep_strict(train_oct)
y_tr   = train_oct[TARGET_COL]

X_val  = prep_strict(valid_nov)
y_val  = valid_nov[TARGET_COL]

X_test = prep_strict(test_dec)
y_true_tn = test_dec['tn']            # tn reales para métricas finales


# ------------------------------------------------------------------
# 4) MODELOS
# ------------------------------------------------------------------
seed = 42
params_base = {
    "objective": "regression",
    "metric": "mae",
    "learning_rate": 0.1,
    "num_leaves": 50,
    "seed": seed,
    "feature_fraction_seed": seed,
    "bagging_seed": seed,
    "nthread": 1,
    "deterministic": True
}

params_tuned = {
    "objective": "regression",
    "metric": "mae",
    "learning_rate": 0.08813259224734658,
    "num_leaves": 79,
    "min_child_samples": 100,
    "feature_fraction": 0.9833497436436693,
    "bagging_fraction": 0.9951622648438583,
    "bagging_freq": 0,
    "lambda_l2": 0.3939212978268534,
    "seed": 42,
    "feature_fraction_seed": 42,
    "bagging_seed": 42,
    "nthread": 1,
    "deterministic": True,
    "verbose": -1
}


base_model_sinLog = train_lgb(X_tr, y_tr, X_val, y_val,
                       params_base, num_rounds=1000, early_stop=50, name='BASE')

tuned_model_sinLog = train_lgb(X_tr, y_tr, X_val, y_val,
                        params_tuned, num_rounds=4000, early_stop=200, name='TUNED')

def wmape(y_true, y_pred, eps=1e-8):
    return np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + eps)


# ------------------------------------------------------------------
# 5) EVALUACIÓN EN DIC-19  (toneladas reales)
# ------------------------------------------------------------------
for tag, model in [("BASE", base_model), ("TUNED", tuned_model)]:
    y_pred_log = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred_tn  = np.expm1(y_pred_log) if USE_LOG_TARGET else y_pred_log

    mae   = mean_absolute_error(y_true_tn, y_pred_tn)
    mape  = mape_safe(y_true_tn, y_pred_tn)
    wmAPE = wmape(y_true_tn, y_pred_tn)

    print(f"{tag:>5} | MAE: {mae:.2f} | MAPE: {mape:.2%} | WMAPE: {wmAPE:.2%}")



▶ BASE | MAE valid (log): 27.2135 | best_iter: 30
[200]	valid's l1: 65.915
▶ TUNED | MAE valid (log): 26.7651 | best_iter: 31
 BASE | MAE: 30.02 | MAPE: 153.81% | WMAPE: 93.12%
TUNED | MAE: 30.01 | MAPE: 159.50% | WMAPE: 93.10%


In [27]:
# ------------------------------------------------------------------
# Loop rápido de p Tweedie y captura de resultados
# ------------------------------------------------------------------
powers = [1.1, 1.2, 1.3, 1.4, 1.5]
results = []

for p in powers:
    params_p = params_base.copy()
    params_p.update({
        "objective": "tweedie",
        "tweedie_variance_power": p,
    })
    model_p1 = train_lgb(
        X_tr, y_tr,
        X_val, y_val,
        params_p,
        num_rounds=4000,
        early_stop=200,
        name=f"TWEED_p{p}"
    )

    # --- eval en dic-19 ---
    y_pred = model_p1.predict(X_test, num_iteration=model_p.best_iteration)
    wm    = wmape(y_true_tn, y_pred)
    results.append((p, wm))
    print(f"p={p:.1f} | WMAPE: {wm:.2%}")

# ------------------------------------------------------------------
# Extraer el mejor p
# ------------------------------------------------------------------
best_p, best_wmape = min(results, key=lambda x: x[1])
print(f"\nMejor p = {best_p:.1f} con WMAPE = {best_wmape:.2%}")

# ------------------------------------------------------------------
# 1) Definimos hiper-parámetros usando best_p
# ------------------------------------------------------------------
params_tweed_opt = params_base.copy()
params_tweed_opt.update({
    "objective":              "tweedie",
    "tweedie_variance_power": best_p,
    # Opcionalmente afina learning_rate, num_leaves, etc.
})

# ------------------------------------------------------------------
# 2) Entrenamos el modelo final con el best_p
# ------------------------------------------------------------------
tweed_opt_model_sinLog = train_lgb(
    X_tr, y_tr,
    X_val, y_val,
    params_tweed_opt,
    num_rounds=4000,
    early_stop=300,
    name=f"TWEED_p{best_p:.1f}"
)

# ------------------------------------------------------------------
# 3) Evaluación comparativa en DIC-19
# ------------------------------------------------------------------
for tag, model in [("BASE SIN LOG",    base_model_sinLog),
                   ("TUNED SIN LOG",   tuned_model_sinLog),
                   (f"TWEED_{best_p:.1f} SIN LOG ", tweed_opt_model_sinLog)]:
    y_pred_log = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred_tn  = np.expm1(y_pred_log) if USE_LOG_TARGET else y_pred_log

    mae   = mean_absolute_error(y_true_tn, y_pred_tn)
    mape  = mape_safe(y_true_tn, y_pred_tn)
    wmAPE = wmape(y_true_tn, y_pred_tn)

    print(f"{tag:>10} | MAE: {mae:.2f} | MAPE: {mape:.2%} | WMAPE: {wmAPE:.2%}")

[200]	valid's l1: 58.7375
▶ TWEED_p1.1 | MAE valid (log): 13.1723 | best_iter: 43
p=1.1 | WMAPE: 54.68%
[200]	valid's l1: 54.152
▶ TWEED_p1.2 | MAE valid (log): 18.3433 | best_iter: 30
p=1.2 | WMAPE: 61.09%
[200]	valid's l1: 56.3038
▶ TWEED_p1.3 | MAE valid (log): 18.3676 | best_iter: 27
p=1.3 | WMAPE: 61.87%
[200]	valid's l1: 51.6523
▶ TWEED_p1.4 | MAE valid (log): 9.1824 | best_iter: 94
p=1.4 | WMAPE: 36.79%
[200]	valid's l1: 55.8633
▶ TWEED_p1.5 | MAE valid (log): 11.5089 | best_iter: 31
p=1.5 | WMAPE: 36.86%

Mejor p = 1.4 con WMAPE = 36.79%
[200]	valid's l1: 51.6523
▶ TWEED_p1.4 | MAE valid (log): 9.1824 | best_iter: 94
BASE SIN LOG | MAE: 29.34 | MAPE: 7480.36% | WMAPE: 91.00%
TUNED SIN LOG | MAE: 27.44 | MAPE: 5874.53% | WMAPE: 85.13%
TWEED_1.4 SIN LOG  | MAE: 7.84 | MAPE: 369.12% | WMAPE: 24.31%


In [28]:
# ------------------------------------------------------------------
# 3)  Evaluación - aca hay que agruegar el modelo neuvo - ya estan los uqe ya entrenamos
# ------------------------------------------------------------------
model_list = [
    ("BASE",   base_model),
    ("TUNED",  tuned_model),
    ("TWEED",  tweed_opt_model),
    ("FINAL",  final_model),
    ("Base Sin log",  base_model_sinLog),
    ("TUNED sin log",  tuned_model_sinLog),
    ("TWEED sin log ",  tweed_opt_model_sinLog)
]

In [29]:

# ------------------------------------------------------------------
# 1)  Capturamos el orden de categorías del entrenamiento
# ------------------------------------------------------------------
cat_levels = {
    c: pd.Categorical(X_trval[c]).categories
    for c in cat_feats
}

def encode_like_train(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c, cats in cat_levels.items():
        if c in out.columns:
            out[c] = (
                pd.Categorical(out[c], categories=cats)
                  .codes.astype("int32")
            )
    return out

# ------------------------------------------------------------------
# 2)  Función WMAPE segura (no será sobrescrita)
# ------------------------------------------------------------------
def wmape_func(y_true, y_pred, eps=1e-8):
    return np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + eps)

In [30]:
# ---------------------------------------------------------------
# 0) IMPORTS
# ---------------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

# ---------------------------------------------------------------
# 1)  Captura los niveles de *todas* las columnas categóricas
# ---------------------------------------------------------------
cat_levels = {}
for c in X_trval.select_dtypes(include='category').columns:
    cat_levels[c] = pd.Categorical(X_trval[c]).categories

def encode_like_train(df: pd.DataFrame) -> pd.DataFrame:
    """Convierte cualquier columna 'category' al código entero
       coherente con el entrenamiento. Crea columnas faltantes = -1."""
    out = df.copy()

    # Convierte cada col categórica
    for c, cats in cat_levels.items():
        if c not in out.columns:
            out[c] = -1                       # si faltaba, todo NA
        out[c] = (
            pd.Categorical(out[c], categories=cats)
              .codes
              .astype("int32")
        )

    # Asegúrate de que no queden columnas 'category'
    cat_cols_left = out.select_dtypes(include='category').columns
    if len(cat_cols_left):
        out[cat_cols_left] = out[cat_cols_left].apply(
            lambda s: s.cat.codes.astype("int32")
        )

    return out

# ---------------------------------------------------------------
# 2)  Métricas auxiliares (por si no las tenías)
# ---------------------------------------------------------------
def mape_safe(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return np.mean(np.abs((y_true - y_pred) /
                          np.maximum(np.abs(y_true), eps)))

def wmape_func(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# ---------------------------------------------------------------
# 3)  Constantes clipping (para modelos entrenados en log)
# ---------------------------------------------------------------
MAX_LOG, MIN_LOG = 20.0, -20.0           # ajusta si hiciera falta

def to_tn(log_pred, needs_expm1):
    if needs_expm1:
        log_pred = np.clip(log_pred, MIN_LOG, MAX_LOG)
        return np.expm1(log_pred)
    return log_pred

# ---------------------------------------------------------------
# 4)  Pre‑codifica X_test una vez
# ---------------------------------------------------------------
X_test_enc = encode_like_train(X_test)

# ---------------------------------------------------------------
# 5)  Evaluación comparativa
# ---------------------------------------------------------------
print("🔍  Evaluación (MAE / MAPE / WMAPE):\n")

for tag, model in model_list:
    cols = model.feature_name()                 # orden original
    X_mat = X_test_enc[cols].to_numpy(dtype=np.float32)

    log_pred = model.predict(X_mat,
                             num_iteration=model.best_iteration or 1)

    needs_expm1 = 'sin log' not in tag.lower()  # aplica sólo a modelos log
    y_pred = to_tn(log_pred, needs_expm1)

    if not np.all(np.isfinite(y_pred)):
        raise ValueError(f"{tag}: NaN o Inf detectados en y_pred")

    mae  = mean_absolute_error(y_true_tn, y_pred)
    mape = mape_safe(y_true_tn, y_pred)
    wmp  = wmape_func(y_true_tn, y_pred)

    print(f"{tag:>16} | MAE: {mae:,.2f} | "
          f"MAPE: {mape:.2%} | WMAPE: {wmp:.2%}")


🔍  Evaluación (MAE / MAPE / WMAPE):

            BASE | MAE: 7.79 | MAPE: 211.39% | WMAPE: 24.15%
           TUNED | MAE: 8.27 | MAPE: 222.78% | WMAPE: 25.65%
           TWEED | MAE: 7.61 | MAPE: 210.64% | WMAPE: 23.60%
           FINAL | MAE: 8.63 | MAPE: 183.18% | WMAPE: 26.76%
    Base Sin log | MAE: 29.22 | MAPE: 7480.32% | WMAPE: 90.65%
   TUNED sin log | MAE: 27.39 | MAPE: 5876.63% | WMAPE: 84.96%
  TWEED sin log  | MAE: 8.26 | MAPE: 386.38% | WMAPE: 25.63%


Agregamos el modelo nuevo a la lista de modelos a comparar

Comparamos los modelos y elegimos cual va a ser el choosen model

### Resutlados para el mejor modelo - Lo levantamos del output anterior

In [31]:
model_list = [
    ("BASE",   base_model),
    ("TUNED",  tuned_model),
    ("TWEED",  tweed_opt_model),
    ("FINAL",  final_model),
    ("Base Sin log",  base_model_sinLog),
    ("TUNED sin log",  tuned_model_sinLog),
    ("TWEED sin log ",  tweed_opt_model_sinLog)
]

# Elemnto clave del pipeline nos permite cambiar a log sin log y usar el mismo codigo

In [32]:
Choosen_model=tuned_model  #agarramos el modelo que queremos y lo igualamos a c
sinlog        = False                   # ← True  si el target NO está en log
                                         # ← False si el modelo devuelve log

In [33]:
# ────────────────────────────────────────────────────────────────────
# 0) Elige modelo y marca si viene SIN LOG
# ────────────────────────────────────────────────────────────────────
Choosen_model = tuned_model   # ← cambia según el que pruebes
sinlog        = False                # ← True  si el target NO está en log
                                    # ← False si el modelo devuelve log‑tn

# ────────────────────────────────────────────────────────────────────
# 1) Prepara X_test alineado al modelo
# ────────────────────────────────────────────────────────────────────
feat_used = Choosen_model.feature_name()
X_eval = X_test[feat_used].copy()

for c in cat_feats:
    if c in X_eval.columns:
        # convierte categorías a códigos numéricos homogéneos
        X_eval[c] = pd.Categorical(X_eval[c]).codes.astype("float32")

X_mat = X_eval.to_numpy(dtype=np.float32)

# ────────────────────────────────────────────────────────────────────
# 2) Predice y convierte a tn reales si procede
# ────────────────────────────────────────────────────────────────────
raw_pred = Choosen_model.predict(X_mat,
                                 num_iteration=Choosen_model.best_iteration or 1)

if sinlog:
    y_pred_tn = raw_pred                     # ya está en tn
else:
    # clipping anti‑overflow (opcional, ajusta límites si lo necesitas)
    raw_pred = np.clip(raw_pred, -20, 20)    # exp(20)-1 ≈ 4.8e8
    y_pred_tn = np.expm1(raw_pred)           # log → tn

y_true = y_true_tn                           # ya definido como test_dec['tn']

# ────────────────────────────────────────────────────────────────────
# 3) Métricas
# ────────────────────────────────────────────────────────────────────
mae   = mean_absolute_error(y_true, y_pred_tn)
wmape = np.sum(np.abs(y_true - y_pred_tn)) / np.sum(np.abs(y_true))

print(f"► Diciembre‑19 | MAE: {mae:.3f} tn | WMAPE: {wmape:.2%}")

# ────────────────────────────────────────────────────────────────────
# 4) DataFrame de predicciones
# ────────────────────────────────────────────────────────────────────
df_preds = pd.DataFrame({
    "product_id": test_dec["product_id"].values,
    "real_tn":    y_true,
    "pred_tn":    y_pred_tn
})
df_preds["abs_error"] = np.abs(df_preds["real_tn"] - df_preds["pred_tn"])

# ────────────────────────────────────────────────────────────────────
# 5) Mostrar primeras 10 filas
# ────────────────────────────────────────────────────────────────────
print(df_preds.head(10))



► Diciembre‑19 | MAE: 8.267 tn | WMAPE: 25.65%
       product_id     real_tn      pred_tn   abs_error
21569       20001  1504.68856  1534.452766   29.764206
21570       20002  1087.30855  1394.285520  306.976970
21571       20003   892.50129  1080.218306  187.717016
21572       20004   637.90002   588.297242   49.602778
21573       20005   593.24443   468.590944  124.653486
21574       20006   417.23228   416.172849    1.059431
21575       20007   390.43432   393.766719    3.332399
21576       20008   195.36854   411.057549  215.689009
21577       20009   495.03574   579.856367   84.820627
21578       20010   359.59998   370.013256   10.413276


sacamos los features asi despues ya los podemos usar

In [34]:
feat_used = Choosen_model.feature_name()   # ← lista de 62 columnas reales

print(feat_used)

['product_id', 'plan_precios_cuidados', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'antiguedad_meses', 'stock_t0', 'month', 'quarter', 'month_sin', 'month_cos', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'lag_15', 'lag_16', 'lag_17', 'lag_18', 'lag_19', 'lag_20', 'lag_21', 'lag_22', 'lag_23', 'lag_24', 'delta_lag_1', 'delta_lag_2', 'delta_lag_3', 'delta_lag_4', 'delta_lag_5', 'delta_lag_6', 'delta_lag_7', 'delta_lag_8', 'delta_lag_9', 'delta_lag_10', 'delta_lag_11', 'delta_lag_12', 'delta_lag_13', 'delta_lag_14', 'delta_lag_15', 'delta_lag_16', 'delta_lag_17', 'delta_lag_18', 'delta_lag_19', 'delta_lag_20', 'delta_lag_21', 'delta_lag_22', 'delta_lag_23', 'tn_media_2', 'tn_media_3', 'tn_media_4', 'tn_media_6', 'tn_media_10', 'flag_stock_disp', 'ratio_tn_stock', 'delta_yoy', 'ratio_yoy']


***Prediccion enero 2020***

Creamos dataset enero

In [35]:
ENE = "2020-01-01"


# ------------------------------------------------------------------
# 4) Crear prototipo enero-20 vacío y concatenar
# ------------------------------------------------------------------
base_cols = df_final.columns.difference(["tn", "cust_request_tn"])
jan_proto = (
    df_final[df_final["periodo"] == "2019-12-01"][base_cols]
      .assign(periodo=pd.to_datetime(ENE),
              tn=np.nan,
              cust_request_tn=np.nan)
)
df_ene = pd.concat([df_final, jan_proto], ignore_index=True)
df_ene["periodo"] = pd.to_datetime(df_ene["periodo"])
# 2) Añade un mes de antigüedad extra
df_ene["antiguedad_meses"] = df_ene["antiguedad_meses"] + 1





Mergeamos con stock


In [36]:
# ------------------------------------------------------------------
# 1) MERGE con stock desplazado +1 mes (stock_t0)
# ------------------------------------------------------------------
stock = (
    df_tb_stocks_02.copy()
        .assign(periodo=lambda d:
                pd.to_datetime(d["periodo"].astype(str), format="%Y%m")
                + pd.offsets.MonthEnd(0)      # 201810 → 2018-10-31
                + pd.offsets.MonthBegin(1))   #        → 2018-11-01
        .rename(columns={"stock_final": "stock_t0"})
)

df_enefeatsandStock = df_ene.merge(
    stock[["product_id", "periodo", "stock_t0"]],
    on=["product_id", "periodo"],
    how="left"
)


Este dataset es el dataset que vamos a usar para predecir enero , con el modelo entrenado para diciembre - el choosen modelo - es decir el mejor modelo en funcion de la comparativa anterior

In [37]:
df_enefeatsandStock.tail()

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,antiguedad_meses,stock_t0
23124,2020-01-01,21263,0,PC,CABELLO,SHAMPOO,VICHY,250.0,NaN,NaN,15,0.50420
23125,2020-01-01,21265,0,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32.0,NaN,NaN,10,0.22068
23126,2020-01-01,21266,0,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32.0,NaN,NaN,10,0.11603
23127,2020-01-01,21267,0,PC,PIEL1,Cara,NIVEA,250.0,NaN,NaN,10,0.54007
23128,2020-01-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,NaN,NaN,10,1.05592


ahora le aplicamos el Feature engineering  

aplicamos el feature engineering

In [38]:
df_eneTuned = feature_pipeline(df_enefeatsandStock)

In [39]:
df_eneTuned

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,...,delta_lag_23,tn_media_2,tn_media_3,tn_media_4,tn_media_6,tn_media_10,flag_stock_disp,ratio_tn_stock,delta_yoy,ratio_yoy
0,2017-01-01,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,937.72717,934.77222,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
496,2017-02-01,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,833.72187,798.01620,...,NaN,934.772220,934.772220,934.772220,934.772220,934.772220,0,NaN,NaN,NaN
994,2017-03-01,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1330.74697,1303.35771,...,NaN,866.394210,866.394210,866.394210,866.394210,866.394210,0,NaN,NaN,NaN
1495,2017-04-01,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1132.94430,1069.96130,...,NaN,1050.686955,1012.048710,1012.048710,1012.048710,1012.048710,0,NaN,NaN,NaN
1995,2017-05-01,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1550.68936,1502.20132,...,NaN,1186.659505,1057.111737,1026.526858,1026.526858,1026.526858,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20008,2019-09-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.01856,0.01856,...,NaN,0.007440,0.018580,0.037143,0.062132,0.062132,1,0.011650,NaN,NaN
20788,2019-10-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.02079,0.02079,...,NaN,0.015605,0.011147,0.018575,0.044810,0.055907,1,0.021158,NaN,NaN
21568,2019-11-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.03341,0.03341,...,NaN,0.019675,0.017333,0.013558,0.031320,0.051518,1,0.019615,NaN,NaN
22348,2019-12-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,0.00892,0.00892,...,NaN,0.027100,0.024253,0.021352,0.021417,0.049506,1,0.031456,NaN,NaN


hacemos cuadrar los features a la fuerza

In [40]:
feat_used_count = len(feat_used)
X_ene_columns_count = len(df_eneTuned.columns)

print(f"Count of columns in feat_used: {feat_used_count}")
print(f"Count of columns in X_ene.columns: {X_ene_columns_count}")

if feat_used_count == X_ene_columns_count:
    print("The number of columns matches!")
else:
    print("The number of columns DOES NOT match.")

Count of columns in feat_used: 69
Count of columns in X_ene.columns: 72
The number of columns DOES NOT match.


In [41]:

def encode_cats(df, cat_feats):
    df2 = df.copy()
    for c in cat_feats:
        if c in df2.columns:
            # pd.Categorical works on any dtype, then .codes → int
            df2[c] = pd.Categorical(df2[c]).codes.astype("int32")
    return df2

# 5.3) Numéricos para train y valid
df_eneTuned = encode_cats(df_eneTuned, cat_feats)

# -------------------------------------------------

In [42]:
df_eneTuned.head()

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,...,delta_lag_23,tn_media_2,tn_media_3,tn_media_4,tn_media_6,tn_media_10,flag_stock_disp,ratio_tn_stock,delta_yoy,ratio_yoy
0,2017-01-01,20001,0,1,10,47,0,3000.0,937.72717,934.77222,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
496,2017-02-01,20001,0,1,10,47,0,3000.0,833.72187,798.01620,...,NaN,934.772220,934.772220,934.772220,934.772220,934.772220,0,NaN,NaN,NaN
994,2017-03-01,20001,0,1,10,47,0,3000.0,1330.74697,1303.35771,...,NaN,866.394210,866.394210,866.394210,866.394210,866.394210,0,NaN,NaN,NaN
1495,2017-04-01,20001,0,1,10,47,0,3000.0,1132.94430,1069.96130,...,NaN,1050.686955,1012.048710,1012.048710,1012.048710,1012.048710,0,NaN,NaN,NaN
1995,2017-05-01,20001,0,1,10,47,0,3000.0,1550.68936,1502.20132,...,NaN,1186.659505,1057.111737,1026.526858,1026.526858,1026.526858,0,NaN,NaN,NaN


Predecimos enero con nuestro mejor modelo

In [43]:
import numpy as np
import pandas as pd

# ────────────────────────────────────────────────────────────────────
# Listas ya definidas
# ────────────────────────────────────────────────────────────────────
DROP_COLS_NOW = [
    'tn',        # target
    'tn_log',    # model target
    'periodo',   # será usado para filtrar antes de borrar
    'cust_request_tn'
]
cat_feats = ['cat1','cat2','cat3','brand','month','quarter']

# ────────────────────────────────────────────────────────────────────
# 1) Codifica todas las categóricas en df_eneTuned
# ────────────────────────────────────────────────────────────────────
def encode_cats(df, cat_feats):
    df2 = df.copy()
    for c in cat_feats:
        if c in df2.columns:
            df2[c] = pd.Categorical(df2[c]).codes.astype("int32")
    return df2

df_eneTuned = encode_cats(df_eneTuned, cat_feats)

# ────────────────────────────────────────────────────────────────────
# 2) Filtra solo enero usando 'periodo'
# ────────────────────────────────────────────────────────────────────
mask_jan = df_eneTuned["periodo"] == ENE
df_jan   = df_eneTuned.loc[mask_jan].copy()

# ────────────────────────────────────────────────────────────────────
# 3) Ahora sí, elimina las columnas que no entran al modelo
# ────────────────────────────────────────────────────────────────────
df_jan_model = df_jan.drop(columns=DROP_COLS_NOW, errors='ignore')

# ────────────────────────────────────────────────────────────────────
# 4) Alinea columnas con las que vio el modelo
# ────────────────────────────────────────────────────────────────────
feat_used = Choosen_model.feature_name()
X_jan     = df_jan_model[feat_used].copy()  # KeyError si falta alguna

# ────────────────────────────────────────────────────────────────────
# 5) Prepara matriz y predice
# ────────────────────────────────────────────────────────────────────
# 5) Prepara matriz y predice
X_mat     = X_jan.to_numpy(dtype=np.float32)
best_it   = getattr(Choosen_model, "best_iteration", 1)
y_predlog = Choosen_model.predict(X_mat, num_iteration=best_it)

# --- APLICAR O NO expm1 SEGÚN EL FLAG ---------------------------------
if sinlog:                                   # ← modelo SIN log
    y_pred_tn = y_predlog                    # ya está en tn reales
    y_predlog = np.log1p(y_predlog)          # (opcional) guardas el log
else:                                        # ← modelo CON log
    y_pred_tn = np.expm1(np.clip(y_predlog, -20, 20))  # log → tn

# ────────────────────────────────────────────────────────────────────
# 6) Añade las predicciones a df_jan
# ────────────────────────────────────────────────────────────────────
df_jan["y_pred_log"] = y_predlog
df_jan["pred_tn"]    = np.expm1(y_predlog)

# ────────────────────────────────────────────────────────────────────
# 7) (Opcional) Inserta de vuelta en el DataFrame completo
# ────────────────────────────────────────────────────────────────────
df_eneTuned.loc[mask_jan, ["y_pred_log","pred_tn"]] = df_jan[["y_pred_log","pred_tn"]]

# ────────────────────────────────────────────────────────────────────
# 8) Vista rápida
# ────────────────────────────────────────────────────────────────────
print(df_jan[["periodo","product_id","y_pred_log","pred_tn"]].head(10))




         periodo  product_id  y_pred_log      pred_tn
22349 2020-01-01       20001    7.274431  1441.930091
22350 2020-01-01       20002    7.106552  1218.933893
22351 2020-01-01       20003    6.578869   718.725056
22352 2020-01-01       20004    6.133498   460.046189
22353 2020-01-01       20005    6.168107   476.281587
22354 2020-01-01       20006    6.081393   436.638242
22355 2020-01-01       20007    5.843246   343.896973
22356 2020-01-01       20008    5.898521   363.498085
22357 2020-01-01       20009    6.035241   416.899676
22358 2020-01-01       20010    5.711465   301.313699


chequeo

In [44]:
ids = [20001, 20002, 20004, 20008]

# Opción habitual
df_filtrado = df_ene[df_ene["product_id"].isin(ids)]
df_filtrado.tail(50)



,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,antiguedad_meses
13564,2019-01-01,20004,0,FOODS,ADEREZOS,Mayonesa,NATURA,240.0,537.66267,511.33713,25
13568,2019-01-01,20008,0,HC,VAJILLA,Opaco,LIMPIEX,750.0,544.38661,543.27828,25
14216,2019-02-01,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1343.99435,1259.09363,26
14217,2019-02-01,20002,0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,1090.26594,1043.01349,26
14219,2019-02-01,20004,0,FOODS,ADEREZOS,Mayonesa,NATURA,240.0,449.98505,441.70332,26
14223,2019-02-01,20008,0,HC,VAJILLA,Opaco,LIMPIEX,750.0,487.10919,476.98787,26
14876,2019-03-01,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,1607.18772,1470.65653,27
14877,2019-03-01,20002,0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,1112.63099,1083.62552,27
14879,2019-03-01,20004,0,FOODS,ADEREZOS,Mayonesa,NATURA,240.0,631.38080,619.77084,27
14883,2019-03-01,20008,0,HC,VAJILLA,Opaco,LIMPIEX,750.0,469.77325,454.57037,27


Asignamos Tn la prediccion

In [45]:
# 6b) Asigna la predicción al campo 'tn' original
df_jan["tn"] = df_jan["pred_tn"]

# Si prefieres hacerlo sobre df_eneTuned y no en la copia df_jan:
df_eneTuned.loc[mask_jan, "tn"] = df_jan["pred_tn"].values


In [46]:
df_jan.head()

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,...,tn_media_3,tn_media_4,tn_media_6,tn_media_10,flag_stock_disp,ratio_tn_stock,delta_yoy,ratio_yoy,y_pred_log,pred_tn
22349,2020-01-01,20001,0,1,10,47,0,3000.0,NaN,1441.930091,...,1487.855463,1530.893000,1510.651745,1492.192550,1,21.570592,18.00187,1.012109,7.274431,1441.930091
22350,2020-01-01,20002,0,1,10,47,15,3000.0,NaN,1218.933893,...,1496.807430,1395.152500,1243.473690,1179.544470,1,6.169744,77.85397,1.077125,7.106552,1218.933893
22351,2020-01-01,20003,0,0,0,48,22,475.0,NaN,718.725056,...,974.053890,972.483208,873.455267,769.662113,1,3.384512,122.67260,1.159351,6.578869,718.725056
22352,2020-01-01,20004,0,0,0,48,22,240.0,NaN,460.046189,...,808.846137,803.177452,702.759787,657.354349,1,0.526160,52.33525,1.089376,6.133498,460.046189
22353,2020-01-01,20005,0,0,0,48,22,120.0,NaN,476.281587,...,732.312970,769.116747,726.480795,724.575737,1,0.774859,220.61015,1.592029,6.168107,476.281587


Agregamos TN el dataset del princpio que usamos para estimar enero antes de pasarlo por el pipeline

In [47]:
df_enefeatsandStock

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,antiguedad_meses,stock_t0
0,2017-01-01,20001,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,937.72717,934.77222,1,NaN
1,2017-01-01,20002,0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,555.18654,550.15707,1,NaN
2,2017-01-01,20003,0,FOODS,ADEREZOS,Mayonesa,NATURA,475.0,1067.81543,1063.45835,1,NaN
3,2017-01-01,20004,0,FOODS,ADEREZOS,Mayonesa,NATURA,240.0,569.37394,555.91614,1,NaN
4,2017-01-01,20005,0,FOODS,ADEREZOS,Mayonesa,NATURA,120.0,494.60084,494.27011,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
23124,2020-01-01,21263,0,PC,CABELLO,SHAMPOO,VICHY,250.0,NaN,NaN,15,0.50420
23125,2020-01-01,21265,0,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32.0,NaN,NaN,10,0.22068
23126,2020-01-01,21266,0,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32.0,NaN,NaN,10,0.11603
23127,2020-01-01,21267,0,PC,PIEL1,Cara,NIVEA,250.0,NaN,NaN,10,0.54007


In [48]:
# Suponemos que df_jan ya tiene ['product_id','pred_tn'] para enero
pred_jan = df_jan[['product_id', 'pred_tn']].rename(columns={'pred_tn': 'tn_pred'})

# Hacemos un left‐join sobre product_id (y opcionalmente periodo si quieres)
df_merged = (
    df_enefeatsandStock
    .merge(pred_jan, on='product_id', how='left')
)

# Sobreescribimos tn con la predicción donde exista
df_merged['tn'] = df_merged['tn_pred'].fillna(df_merged['tn'])

# (Opcional) Eliminamos la columna auxiliar
df_merged = df_merged.drop(columns=['tn_pred'])

# Resultado
print(df_merged[['periodo','product_id','tn']].head())



     periodo  product_id           tn
0 2017-01-01       20001  1441.930091
1 2017-01-01       20002  1218.933893
2 2017-01-01       20003   718.725056
3 2017-01-01       20004   460.046189
4 2017-01-01       20005   476.281587


In [49]:
    df_merged.tail()

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,antiguedad_meses,stock_t0
23124,2020-01-01,21263,0,PC,CABELLO,SHAMPOO,VICHY,250.0,NaN,0.640368,15,0.50420
23125,2020-01-01,21265,0,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32.0,NaN,0.488171,10,0.22068
23126,2020-01-01,21266,0,PC,PIEL1,CUIDADO ESPECIAL,LANCOME,32.0,NaN,0.488171,10,0.11603
23127,2020-01-01,21267,0,PC,PIEL1,Cara,NIVEA,250.0,NaN,0.541683,10,0.54007
23128,2020-01-01,21276,0,PC,PIEL1,Cara,NIVEA,140.0,NaN,0.541683,10,1.05592


In [50]:
# Guarda el DataFrame en CSV
df_merged.to_csv('/content/sample_data/df_predenero_optuna.csv', index=False)
print("✅ CSV guardado en /content/sample_data/df_predenero_optuna.csv")


✅ CSV guardado en /content/sample_data/df_predenero_optuna.csv


**Prediccion Febrero**

A partir del dataset mergeado - Enero mas predicciones de enero - repetimos el proceso para febrero

In [51]:
FEB = "2020-02-01"


# ------------------------------------------------------------------
# 4) Crear prototipo febero-20 vacío y concatenar
# ------------------------------------------------------------------
base_cols = df_merged.columns.difference(["tn", "cust_request_tn"])
feb_proto = (
    df_merged[df_merged["periodo"] == "2020-01-01"][base_cols]
      .assign(periodo=pd.to_datetime(FEB),
              tn=np.nan,
              cust_request_tn=np.nan)
)
df_feb= pd.concat([df_merged, feb_proto], ignore_index=True)
df_feb["periodo"] = pd.to_datetime(df_feb["periodo"])
# 2) Añade un mes de antigüedad extra
df_feb["antiguedad_meses"] = df_feb["antiguedad_meses"] + 1


como necesitamos stcok le ponemos el promedio historico para el mes de febrero

In [52]:

# Constante de febrero
FEB = pd.to_datetime("2020-02-01")

# 1) Calcula el stock_t0 promedio histórico por producto para mes=2
mask_feb_hist = df_merged["periodo"].dt.month == 2
avg_stock = (
    df_merged.loc[mask_feb_hist]
             .groupby("product_id")["stock_t0"]
             .mean()
             .rename("stock_t0_avg")
)

# 2) Asigna ese promedio a las filas de febrero en df_feb
is_feb = df_feb["periodo"] == FEB

# Usa map para mantener el índice
df_feb.loc[is_feb, "stock_t0"] = (
    df_feb.loc[is_feb, "product_id"]
          .map(avg_stock)
)

# 3) Comprueba
print("Febrero filas:", is_feb.sum())
print(df_feb.loc[is_feb, ["product_id","stock_t0"]].head())


Febrero filas: 780
       product_id   stock_t0
23129       20001  544.71331
23130       20002   32.57674
23131       20003   84.65832
23132       20004  221.90114
23133       20005  401.20604


In [53]:
df_feb.tail(25)

,periodo,product_id,plan_precios_cuidados,cat1,cat2,cat3,brand,sku_size,cust_request_tn,tn,antiguedad_meses,stock_t0
23884,2020-02-01,21201,0,PC,DENTAL,Cepilllo,COLGATE,1.0,NaN,NaN,16,0.00527
23885,2020-02-01,21202,0,REF,TE,Hierbas,TWININGS,20.0,NaN,NaN,37,0.85398
23886,2020-02-01,21207,0,PC,DENTAL,Cepilllo,COLGATE,2.0,NaN,NaN,27,1.27715
23887,2020-02-01,21209,0,PC,PIEL1,Limpieza,SKIN1,150.0,NaN,NaN,17,2.85199
23888,2020-02-01,21212,0,HC,HOGAR,Esponja,MUSCULO,1.0,NaN,NaN,27,0.32040
23889,2020-02-01,21214,0,PC,DEOS,RollOn,NIVEA,50.0,NaN,NaN,5,NaN
23890,2020-02-01,21218,0,REF,TE,Hierbas,TWININGS,20.0,NaN,NaN,37,1.12167
23891,2020-02-01,21222,0,REF,TE,Verde,TWININGS,20.0,NaN,NaN,37,0.85904
23892,2020-02-01,21224,0,PC,DEOS,Aero,DEOS3,50.0,NaN,NaN,36,0.44190
23893,2020-02-01,21226,0,PC,PIEL1,Limpieza,SKIN1,25.0,NaN,NaN,17,0.16501


aplicamos feature engineering a deuvelta - lo tenemos que hacer devuelta sobre todo el dataframe si no las los deltas y los lags se rompen , es por eso que todo el fine tunning se  hace nuevamente

In [54]:
df_febTuned = feature_pipeline(df_feb)

In [55]:
df_febTuned.columns

Index(['periodo', 'product_id', 'plan_precios_cuidados', 'cat1', 'cat2',
       'cat3', 'brand', 'sku_size', 'cust_request_tn', 'tn',
       'antiguedad_meses', 'stock_t0', 'month', 'quarter', 'month_sin',
       'month_cos', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6',
       'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13',
       'lag_14', 'lag_15', 'lag_16', 'lag_17', 'lag_18', 'lag_19', 'lag_20',
       'lag_21', 'lag_22', 'lag_23', 'lag_24', 'delta_lag_1', 'delta_lag_2',
       'delta_lag_3', 'delta_lag_4', 'delta_lag_5', 'delta_lag_6',
       'delta_lag_7', 'delta_lag_8', 'delta_lag_9', 'delta_lag_10',
       'delta_lag_11', 'delta_lag_12', 'delta_lag_13', 'delta_lag_14',
       'delta_lag_15', 'delta_lag_16', 'delta_lag_17', 'delta_lag_18',
       'delta_lag_19', 'delta_lag_20', 'delta_lag_21', 'delta_lag_22',
       'delta_lag_23', 'tn_media_2', 'tn_media_3', 'tn_media_4', 'tn_media_6',
       'tn_media_10', 'flag_stock_disp', 'ratio_tn_stock', 'delta

In [56]:
import numpy as np
import pandas as pd

# ────────────────────────────────────────────────────────────────────
# Listas ya definidas (idénticas a las de enero)
# ────────────────────────────────────────────────────────────────────
DROP_COLS_NOW = [
    'tn',        # target
    'tn_log',    # model target
    'periodo',   # será usado para filtrar antes de borrar
    'cust_request_tn'
]
cat_feats = ['cat1','cat2','cat3','brand','month','quarter']

# Constante de febrero
FEB = pd.to_datetime("2020-02-01")

# ────────────────────────────────────────────────────────────────────
# 1) Codifica todas las categóricas en df_febTuned
# ────────────────────────────────────────────────────────────────────
def encode_cats(df, cat_feats):
    df2 = df.copy()
    for c in cat_feats:
        if c in df2.columns:
            df2[c] = pd.Categorical(df2[c]).codes.astype("int32")
    return df2

df_febTuned = encode_cats(df_febTuned, cat_feats)

# ────────────────────────────────────────────────────────────────────
# 2) Filtra solo febrero usando 'periodo'
# ────────────────────────────────────────────────────────────────────
mask_feb = df_febTuned["periodo"] == FEB
df_feb   = df_febTuned.loc[mask_feb].copy()

# ────────────────────────────────────────────────────────────────────
# 3) Elimina las columnas que no entran al modelo
# ────────────────────────────────────────────────────────────────────
df_feb_model = df_feb.drop(columns=DROP_COLS_NOW, errors='ignore')

# ────────────────────────────────────────────────────────────────────
# 4) Alinea columnas con las que vio el modelo
# ────────────────────────────────────────────────────────────────────
feat_used = Choosen_model.feature_name()
X_feb     = df_feb_model[feat_used].copy()  # KeyError si falta alguna

# 5) Prepara matriz y predice
X_mat     = X_feb.to_numpy(dtype=np.float32)
best_it   = getattr(Choosen_model, "best_iteration", 1)
y_predlog = Choosen_model.predict(X_mat, num_iteration=best_it)

# ── decide si aplicar expm1 ───────────────────────────────────────
if sinlog:                                   # modelo SIN log
    y_pred_tn   = y_predlog                  # ya está en tn reales
    y_predlog   = np.log1p(y_predlog)        # (opcional) guardas el log
else:                                        # modelo CON log
    y_pred_tn   = np.expm1(np.clip(y_predlog, -20, 20))  # log → tn

# 6) Añade las predicciones a df_feb
df_feb["y_pred_log"] = y_predlog
df_feb["pred_tn"]    = y_pred_tn
# ────────────────────────────────────────────────────────────────────
# 7) (Opcional) Inserta de vuelta en el DataFrame completo df_febTuned
# ────────────────────────────────────────────────────────────────────
df_febTuned.loc[mask_feb, ["y_pred_log","pred_tn"]] = df_feb[["y_pred_log","pred_tn"]]

# ────────────────────────────────────────────────────────────────────
# 8) Vista rápida
# ────────────────────────────────────────────────────────────────────
print(df_feb[["periodo","product_id","y_pred_log","pred_tn"]].head(10))


         periodo  product_id  y_pred_log      pred_tn
23129 2020-02-01       20001    7.224194  1371.232031
23130 2020-02-01       20002    7.095584  1205.627238
23131 2020-02-01       20003    6.367248   581.452818
23132 2020-02-01       20004    5.951243   383.230831
23133 2020-02-01       20005    5.994154   400.077133
23134 2020-02-01       20006    5.943843   380.397810
23135 2020-02-01       20007    5.746490   312.089713
23136 2020-02-01       20008    5.847383   345.326812
23137 2020-02-01       20009    6.008433   405.845452
23138 2020-02-01       20010    5.725549   305.601536


In [57]:
import pandas as pd

# Constante de febrero
FEB = pd.to_datetime("2020-02-01")

# 1) Filtra solo las filas de febrero
df_submit = df_febTuned.loc[df_febTuned["periodo"] == FEB, ["product_id", "pred_tn"]].copy()

# 2) Renombra la columna de predicción al nombre que exige Kaggle (por ejemplo "tn")
df_submit = df_submit.rename(columns={"pred_tn": "tn"})

# 3) Exporta a CSV delimitado por comas
output_path = '/content/sample_data/df_kaggle_febrero_tuned_model.csv'
df_submit.to_csv(output_path, index=False)

print(f"✅ Archivo de submission generado en: {output_path}")


✅ Archivo de submission generado en: /content/sample_data/df_kaggle_febrero_tuned_model.csv


Exportacion final

In [58]:
import pandas as pd

# 1️⃣ – Renombra la columna `tn` del segundo dataframe
df_submit_renamed = df_submit.rename(columns={'tn': 'tn_feb'})

# 2️⃣ – Une los dos dataframes por `product_id`
df_merged = (
    df_preds
      .merge(df_submit_renamed, on='product_id', how='left')  # «left» conserva todas las filas de df_preds
      .loc[:, ['product_id', 'real_tn', 'pred_tn','tn_feb']]  # orden opcional de columnas
)

# 3️⃣ – Guarda el resultado en CSV (delimitado por comas)
output_path = '/content/sample_data/df_train+tuned_model.csv'
df_merged.to_csv(output_path, index=False, float_format='%.6f')  # float_format es opcional

print(f"✅ Archivo generado en: {output_path}")


✅ Archivo generado en: /content/sample_data/df_train+tuned_model.csv
